In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from matplotlib.colors import LinearSegmentedColormap
import re
import glob
from variables_config import (
    param_info_file, plots_dir, redshift_values, lf_data_dir,
    colour_data_dir, uvlf_limits, colour_limits,
    colour_nbins, filters
)

def read_colour(filename):
    """Read data from file."""
    try:
        data = pd.read_csv(filename, delim_whitespace=True)
        return data
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

def calculate_parameter_values(param_info):
    """Calculate all parameter values based on LogFlag."""
    min_val = param_info['MinVal']
    max_val = param_info['MaxVal']
    fid_val = param_info['FiducialVal']
    
    if param_info['LogFlag']:
        # Logarithmic spacing
        log_min = np.log10(min_val)
        log_max = np.log10(max_val)
        log_fid = np.log10(fid_val)
        
        n1_val = 10**(log_min + (log_fid - log_min)/2)
        val_1 = 10**(log_fid + (log_max - log_fid)/2)
    else:
        # Linear spacing
        n1_val = min_val + (fid_val - min_val)/2
        val_1 = fid_val + (max_val - fid_val)/2
    
    return {
        'n2': min_val,    # MinVal
        'n1': n1_val,     # Calculated intermediate value
        '0': fid_val,     # FiducialVal
        '1': val_1,       # Calculated intermediate value
        '2': max_val      # MaxVal
    }

def get_safe_name(name, filter_system_only=False):
    """
    Convert string to path-safe version and/or extract filter system.
    
    Args:
        name: String to process (e.g., "GALEX FUV" or "UV1500")
        filter_system_only: If True, returns only the filter system (e.g., "GALEX" or "UV")
    
    Returns:
        Processed string (e.g., "GALEX_FUV" or "GALEX")
    """
    # Replace spaces with underscores
    safe_name = name.replace(' ', '_')
    
    # If we only want the filter system, return the first part
    if filter_system_only:
        return safe_name.split('_')[0]
    
    return safe_name

In [2]:
def plot_all_uvlf_variations(param_num, param_info, redshift, band_type, band):
    """Plot UVLF for all variations of a parameter at one redshift for a specific band."""
    param_values = calculate_parameter_values(param_info)
    
    plt.figure(figsize=(10, 6))
    variations = ['n2', 'n1', '0', '1', '2']
    colors = ['blue', 'green', 'black', 'red', 'purple']
    
    # Get filter system from band name
    filter_system = get_safe_name(band, filter_system_only=True)
    
    # Get base output directory
    base_output_dir = lf_data_dir[band_type][filter_system]
    
    # Plot each variation
    for var, color in zip(variations, colors):
        # Use same filename pattern as data files
        filename = os.path.join(get_safe_name(base_output_dir), redshift['label'],
                              f"UVLF_1P_p{param_num}_{var}_{get_safe_name(band)}_{redshift['label']}_{band_type}.txt")
        
        if os.path.exists(filename):
            data = pd.read_csv(filename, delimiter='\t')
            
            label = f'{param_info["ParamName"]} = {param_values[var]:.3g}'
            if var == '0':
                label += ' (fiducial)'
            
            plt.errorbar(data['magnitude'], data['phi'], 
                        yerr=data['phi_sigma'],
                        fmt='o-', color=color, 
                        label=label,
                        markersize=4, capsize=2)
        else:
            print(f"File not found: {filename}")

    # Only save plot if data was actually plotted
    if plt.gca().has_data():
        plt.xlabel('Absolute Magnitude (AB)', fontsize=12)
        plt.ylabel('Number Density (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        plt.xlim(*uvlf_limits)
        plt.grid(True, which='both', linestyle='--', alpha=0.6)
        plt.title(f'UV Luminosity Function - {param_info["ParamName"]} (p{param_num})\n'
                 f'{param_info["Description"]} (z={redshift["redshift"]})',
                 fontsize=14)
        plt.legend(loc='upper left', title='Parameter Values')
        plt.figtext(0.02, 0.02, f'LogFlag: {param_info["LogFlag"]}', fontsize=8)
        
        # Update plot filename to match data pattern
        output_path = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band), redshift['label'],
                                  f'UVLF_1P_p{param_num}_{get_safe_name(band)}_{redshift["label"]}_{band_type}.pdf')
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        print(f"Saving UVLF plot: {output_path}")
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
    
    plt.close()

def plot_all_color_variations(param_num, param_info, redshift, band_type, color_pairs):
    """Plot color distribution for all variations at a given redshift."""
    param_values = calculate_parameter_values(param_info)
    variations = ['n2', 'n1', '0', '1', '2']
    colors = ['blue', 'green', 'black', 'red', 'purple']
    
    # Process each color pair
    for band1, band2 in color_pairs:
        plt.figure(figsize=(10, 6))
        
        # Get filter system from first band
        filter_system = get_safe_name(band1, filter_system_only=True)
        
        for var, color in zip(variations, colors):
            # Create color name for filename using same pattern as data
            color_name = f"{get_safe_name(band1).split('_')[-1]}-{get_safe_name(band2).split('_')[-1]}"
            
            filename = os.path.join(get_safe_name(colour_data_dir[band_type][filter_system]), 
                                  redshift['label'],
                                  f"Colour_1P_p{param_num}_{var}_{color_name}_{redshift['label']}_{band_type}.txt")
            
            if os.path.exists(filename):
                data = pd.read_csv(filename, delimiter='\t')
                
                label = f'{param_info["ParamName"]} = {param_values[var]:.3g}'
                if var == '0':
                    label += ' (fiducial)'
                
                plt.hist(data['colour'], weights=data['distribution'],
                        bins=np.linspace(*colour_limits, colour_nbins + 1),
                        alpha=0.5, color=color, label=label,
                        edgecolor='black', linewidth=1)
            else:
                print(f"File not found: {filename}")
        
        plt.xlabel(f'{band1} - {band2} [mag]')
        plt.ylabel('Normalized Count')
        plt.xlim(*colour_limits)
        plt.grid(True, which='both', linestyle='--', alpha=0.3)
        plt.title(f'{band_type.capitalize()} Color Distribution - {param_info["ParamName"]} (p{param_num})\n'
                 f'{param_info["Description"]} (z={redshift["redshift"]})')
        plt.legend(title='Parameter Values')
        plt.figtext(0.02, 0.02, f'LogFlag: {param_info["LogFlag"]}', fontsize=8)
        
        # Update plot filename to match data pattern
        output_path = os.path.join(plots_dir, 'colours', band_type, filter_system, redshift['label'],
                                  f'Colour_1P_p{param_num}_{color_name}_{redshift["label"]}_{band_type}.pdf')
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        print(f"Saving color plot: {output_path}")
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
        plt.close()

In [3]:
def process_all_parameters():
    """Process all parameters."""
    # Define all color pairs you want to process
    color_pairs = [
        ("GALEX FUV", "GALEX NUV"),
        # Add other pairs as needed
    ]
    
    params_df = pd.read_csv(param_info_file)
    
    for param_num in range(1, 29):
        param_info = params_df.iloc[param_num-1]
        print(f"\nProcessing parameter {param_num}: {param_info['ParamName']}")
        
        # Process each redshift
        for snap, redshift in redshift_values.items():
            print(f"Processing redshift {redshift['redshift']}")
            
            # Plot UVLFs for each band type and filter
            for band_type in ['intrinsic', 'attenuated']:
                for band in filters[band_type]:
                    print(f"Plotting UVLF for {band_type} {band}")
                    plot_all_uvlf_variations(param_num, param_info, redshift, band_type, band)
                
                # Process colors
                print(f"Processing color distributions for z={redshift['redshift']}")
                plot_all_color_variations(param_num, param_info, redshift, band_type, color_pairs)

if __name__ == "__main__":
    process_all_parameters()


Processing parameter 1: Omega0
Processing redshift 2.0
Plotting UVLF for intrinsic UV1500
Saving UVLF plot: /disk/xray15/aem2/plots/28pams/IllustrisTNG/1P/LFs/intrinsic/UV1500/z2.0/UVLF_1P_p1_UV1500_z2.0_intrinsic.pdf
Plotting UVLF for intrinsic GALEX FUV
Saving UVLF plot: /disk/xray15/aem2/plots/28pams/IllustrisTNG/1P/LFs/intrinsic/GALEX_FUV/z2.0/UVLF_1P_p1_GALEX_FUV_z2.0_intrinsic.pdf
Plotting UVLF for intrinsic GALEX NUV
Saving UVLF plot: /disk/xray15/aem2/plots/28pams/IllustrisTNG/1P/LFs/intrinsic/GALEX_NUV/z2.0/UVLF_1P_p1_GALEX_NUV_z2.0_intrinsic.pdf
Processing color distributions for z=2.0
Saving color plot: /disk/xray15/aem2/plots/28pams/IllustrisTNG/1P/colours/intrinsic/GALEX/z2.0/Colour_1P_p1_FUV-NUV_z2.0_intrinsic.pdf
Plotting UVLF for attenuated GALEX FUV
Saving UVLF plot: /disk/xray15/aem2/plots/28pams/IllustrisTNG/1P/LFs/attenuated/GALEX_FUV/z2.0/UVLF_1P_p1_GALEX_FUV_z2.0_attenuated.pdf
Plotting UVLF for attenuated GALEX NUV
Saving UVLF plot: /disk/xray15/aem2/plots/28pam